# QSPRpred API Integration with LangChain Agent

This notebook demonstrates the integration of the BridgeDB API with a LangChain-powered AI agent for biological identifier mapping.

## Table of Contents
1. Setup and Imports
2. Papyrus API Wrapper
3. Custom LangChain Tool
4. AI Agent Configuration
5. Testing the Agent
6. Additional Utility Functions
7. Direct API Calls

## 1. Setup and Imports

First, let's set up our environment and import the necessary libraries.

In [2]:
# Import statements
import os
import requests
import pandas as pd
from dotenv import load_dotenv
from langchain_community.tools import BaseTool
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

from qsprpred.data.sources.papyrus import Papyrus
import qsprpred

# Load environment variables
load_dotenv()

# Set OpenAI API Key
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("OpenAI API Key is not set. Please check your .env file.")

os.environ["OPENAI_API_KEY"] = api_key

dataset_name = "PapyrusTutorialDataset"  # name of the file to be generated
papyrus_version = "latest"  # Papyrus database version
data_dir = "papyrus"  # directory to store the Papyrus data
output_dir = "data"  # directory to store the generated dataset

In [ ]:
# from papyrus_scripts.download import download_papyrus
# download_papyrus(version='latest', structures=True, descriptors=['mold2', 'unirep'])

### Practice dataset

First we'll create a test dataset for validating the results for the dataset retriever

In [3]:
from qsprpred.data.sources.papyrus import Papyrus
import qsprpred

acc_keys = ["P29274"]
dataset_name = "PapyrusTutorialDataset"  # name of the file to be generated
quality = "high"  # choose minimum quality from {"high", "medium", "low"}
papyrus_version = "latest"  # Papyrus database version
data_dir = "papyrus"  # directory to store the Papyrus data
output_dir = "data"  # directory to store the generated dataset

# Create a Papyrus object, which specifies the version and directory to store the payrus data
papyrus = Papyrus(
    data_dir=data_dir,
    version=papyrus_version,
    stereo=False,
    plus_only=True,
)

# Create subset of payrus data for the given accession keys, returns a MoleculeTable
mt = papyrus.getData(
    dataset_name,
    acc_keys,
    quality,
    output_dir=output_dir,
    use_existing=False,
    activity_types=["Ki", "IC50", "Kd"]
)
mt.getDF().head()

len(mt.getDF())

0it [00:00, ?it/s]

3785

## 2. QSPRpred API Wrapper

We'll create a wrapper class for the QSPRpred API to handle identifier mapping requests.

In [4]:
# PapyrusAPI Class
# This class encapsulates the functionality to interact with the Papyrus database:

class PapyrusAPI:
    papyrus_version = "latest"  # Papyrus database version
    data_dir = "papyrus"  # directory to store the Papyrus data
    output_dir = "data"  # directory to store the generated dataset


    @staticmethod
    def fetch_data(acc_keys, quality):
        papyrus = Papyrus(
                data_dir=data_dir,
                version=papyrus_version,
                stereo=False,
                plus_only=True,
            )
        # create subset
        mt = papyrus.getData(
                dataset_name,
                acc_keys,
                quality,
                output_dir=output_dir,
                use_existing=False,
                activity_types=["Ki", "IC50", "Kd"]
            )
        if type(mt) is qsprpred.data.tables.mol.MoleculeTable:
            return mt
        else:
             f"Error: fetched data is {type(mt)}, should be MoleculeTable "

## 3. Custom LangChain Tool

Now, let's create a custom LangChain tool that uses our BridgeDbAPI wrapper.

In [5]:
from qsprpred.data import QSPRDataset, RandomSplit
from qsprpred.data.descriptors.fingerprints import MorganFP
from qsprpred.models import SklearnModel, SklearnMetrics
from sklearn.ensemble import RandomForestRegressor
from rdkit import Chem

# define papyrus dataset handler   

class PapyrusDatasetThresholdTool(BaseTool):
    name: str ="dataset_threshold"
    description: str = "Used for determining if length of Papyrus dataset is higher than threshold"

    def _run(self, query: str) -> str:
        # Parse the query; now expecting only four parts: acc_keys, quality, threshold, smiles
        parts = query.split(",")
        if len(parts) != 4:
            return "Error: Query should be in the format 'acc_keys, quality, threshold, smiles"
        acc_keys, quality, threshold, smiles = [p.strip() for p in parts]
        
        if not Chem.MolFromSmiles(smiles):
            return f"Invalid SMILES found: {smiles}"
        # Attempt to determine if number of rows is higher than threshold
        try:
            papyrus_set = PapyrusAPI().fetch_data(acc_keys, quality)
            num_rows = len(papyrus_set)
            # return if number of rows is higher than threshold
            if num_rows > float(threshold):
                # calculate compound features and split dataset into train and test

                dataset = QSPRDataset.fromMolTable(
                    papyrus_set,
                    name="AssessmentTutorialDataset",
                    target_props=[{"name": "pchembl_value_Mean", "task": "REGRESSION"}],
                    random_state=42
                )

                dataset.prepareDataset(
                    split=RandomSplit(test_fraction=0.2, dataset=papyrus_set),
                    feature_calculators=[MorganFP(radius=3, nBits=2048)],
                    recalculate_features=True,
                )
                
                # Initialize random forest it with the SklearnModel class
                model = SklearnModel(
                    base_dir="models",
                    alg=RandomForestRegressor,
                    name="AgentTestModel",
                )

                # Finally, we need to fit the model on the complete dataset if we want to use it further
                model.fitDataset(dataset)

                # predict the potency of compound based on SMILES
                predictions = model.predictMols([smiles])
                return predictions
            # print(result.getDF().Quality.value_counts())
        except Exception as e:
            return f"Error: {str(e)}"

    def _arun(self, query: str) -> str:
        # Async implementation (not needed for this tool)
        raise NotImplementedError("This tool does not support async")
    
tools = [PapyrusDatasetThresholdTool()]

In [7]:
# Test with a different ENSG identifier (e.g., BRCA2)
tool = PapyrusDatasetThresholdTool()
tool._run("P29275, high, 1000, CCCCCC")

0it [00:00, ?it/s]

array([[6.26884]])

## 4. AI Agent Configuration

Let's set up our AI agent using the custom tool we created.

In [6]:
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.messages import HumanMessage

def create_bridgedb_agent():
    model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    tools = [PapyrusDatasetThresholdTool()]
    memory = MemorySaver()
    agent_executor = create_react_agent(model, tools, checkpointer=memory)
    return agent_executor

papyrus_agent = create_bridgedb_agent()

def run_agent_query(query):
    config = {"configurable": {"thread_id": "qspr_conversation"}}
    
    print(f"Query: {query}\n")
    for chunk in papyrus_agent.stream(
        {"messages": [HumanMessage(content=query)]},
        config
    ):
        if 'agent' in chunk and 'messages' in chunk['agent']:
            for message in chunk['agent']['messages']:
                if hasattr(message, 'content') and message.content:
                    print(message.content)
        print("----")
    print("\n")

# Test queries

run_agent_query("If the number of rows is higher than 1000 for papyrus_set with acc key P29275 and quality high, predict the potency of a compound with smiles CCCCCC")

Query: If the number of rows is higher than 1000 for papyrus_set with acc key P29275 and quality high, predict the potency of a compound with smiles CCCCCC

----
----
----


0it [00:00, ?it/s]

----
The number of rows in the Papyrus dataset for the specified conditions (acc key P29275 and quality high) is higher than 1000. 

Now, I will proceed to predict the potency of the compound with the SMILES representation "CCCCCC".
----


0it [00:00, ?it/s]

----
The predicted potency of the compound with the SMILES representation "CCCCCC" is approximately 6.23.
----


